# High-level RNN Keras (CNTK) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, CuDNNGRU
from common.params_lstm import *
from common.utils import *

Using CNTK backend


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.8
Numpy:  1.13.3
CNTK:  2.1
cntk


In [3]:
def create_symbol(CUDNN=True):
    model = Sequential()
    model.add(Embedding(MAXFEATURES, EMBEDSIZE, input_length=MAXLEN))
    # Only return last output
    if not CUDNN:
        model.add(GRU(NUMHIDDEN, return_sequences=False, return_state=False))
    else:
        model.add(CuDNNGRU(NUMHIDDEN, return_sequences=False, return_state=False))
    model.add(Dense(2, activation='softmax'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.Adam(LR, BETA_1, BETA_2, EPS),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 int32 int32
CPU times: user 5.82 s, sys: 404 ms, total: 6.22 s
Wall time: 7.34 s


In [6]:
%%time
# Load symbol
# CuDNN RNNs are only available with the TensorFlow backend.
sym = create_symbol(CUDNN=False)

CPU times: user 232 ms, sys: 296 ms, total: 528 ms
Wall time: 489 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 6.84 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 125)          3750000   
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               67800     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,818,002
Trainable params: 3,818,002
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/3
25000/25000 [==============================] - ETA: 0s - loss: 0.5332 - acc: 0.7078
Epoch 2/3
25000/25000 [==============================] - ETA: 0s - loss: 0.3421 - acc: 0.8527
Epoch 3/3
25000/25000 [==============================] - ETA: 0s - loss: 0.1511 - acc: 0.9469
CPU times: user 2min 58s, sys: 28.8 s, total: 3min 27s
Wall time: 3min 26s


In [10]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 28.3 s, sys: 4.68 s, total: 32.9 s
Wall time: 33 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.86124
